In [ ]:
import sys
!{sys.executable} -m pip install -U numpy matplotlib scapy[basic] tqdm

In [ ]:
import sys
import threading
from scapy.all import rdpcap
from tqdm.notebook import tqdm

try:  # necessary on Windows, other systems pass
    sys.setrecursionlimit(100000)
    threading.stack_size(200000000)
except:
    pass

def load_captures(files):
    result = []
    
    for file in tqdm(files):
        # use threading to increase stack size
        thread = threading.Thread(target=lambda: result.append(rdpcap(file)))
        thread.start()
        thread.join()
        
    return result

In [ ]:
files = ('../probes/capture-0-a4.pcap', 
         '../probes/capture-1-a4.pcap', 
         '../probes/capture-2-a4.pcap')

(commute_16122019, 
 commute_17122019,
 commute_18122019) = load_captures(files)

In [ ]:
from scapy.all import Dot11

timestamps = []

for pkt in commute_16122019:
    if pkt.haslayer(Dot11) and pkt.type == 0 and pkt.subtype == 4:
        timestamps.append(pkt.time)